# ENLACES

## ¿Qué es un archivo en linux?
En linux un archivo no es **un nombre con datos**

En realidad hay tres cosas distintas:
```css
[nombre] -> [INODO] -> [DATOS]
```

### a. El nombre
  * Vive en un directorio
  * Es solo una etiqueta
  * No contiene datos

### b. El inodo (inode)
Es el corazón del archivo. Contiene:
  * Permisos
  * Dueño y grupo
  * Tamaño
  * Fechas
  * Punteros a los bloques de datos
  * Contador de enlaces (link count)

> El inodo NO tiene nombre

### c. Los datos
  * Bloques en disco
  * El contenido real


Visual:
```yaml
/home/user/archivo.txt
        |
        v
     Inodo 12345
        |
        v
   Bloques de datos
```
> El nombre apunta al inodo, no a los datos directamente


## ¿Qué es un directorio realmente?
Un directoio es una tabla:
| Nombre      | Inodo |
| ----------- | ----- |
| archivo.txt | 12345 |
| foto.jpg    | 67890 |

Nada más.

> El sistema no sabe que "archivo.txt" es especial.

Es solo una fila más.

## ENLACES DUROS | HARD LINK
Un **enlace duro** es otro nombre que apunta al mismo inodo

  * No es enlace directo
  * No es una copia
  * NO apunta al archivo

> Es el archivo.



## ENLACES SIMBÓLICOS | SOFT LINK / SYMLINK
Cambia el modelo.

Un **enlace simbólico** es un archivo especial que contiene una RUTA.

No apunta al inodo del destino, apunta a un `string`.


## DATO
  * Enlace duro = otro nombre del mismo archivo (Inodo)
  * Enlace simbólico = archivo que apunta a otro archivo (Ruta)









> Primero haber revisado [Comando `ln`](../commands/level-3/0-ln.ipynb) para seguir con procesos

## ***ENLACES Y PROCESOS ABIERTOS***
En Linux, un proceso NO trabaja con nombres de archivo.

Trabaja con: 
```bash
Proceso -> descriptor de archivo (fd) -> inodo -> datos
```

El nombre solo se usa al abrir el archivo.
Después desaparece de la ecuación.


### Flujo real
Cuando un proceso hace:
```c
fd = open("archivo.txt", O_RDONLY);
```

El kernel hace:
  1. Busca "archivo.txt" en el directorio
  2. Encuentra el inodo
  3. Crea un descriptor de archivo (fd)
  4. fd apunta al inodo
  5. El proceso ya no depende del nombre

Visual:
```yaml
archivo.txt → inodo 1001 → datos
                   ^
                   |
              fd = 3 (por ejemplo)
```
### ¿Qué pasa si borras un archivo? | HARD LINKS
Ejemplo: 
```bash
ln archivo.txt duro.txt
```
Inodos:
```yaml
archivo.txt -> inodo 1001
duro.txt    -> inodo 1001
```
Proceso abre `archivo.txt`

Luego: 

```bash
rm archivo.txt
```
¿Problema? : Ninguno

Porque:
```bash
duro.txt -> inodo 1001
link count = 1
```
  * Sigue teniendo nombre
  * Sigue teniendo datos
  * Sigue abierto

Si borras todos los nombres: `rm archivo.txt duro.txt`

Ahora:
```yaml
link count = 0
open fd = 1
```

> Sigue vivo hasta que el proceso termine.


### ¿Qué pasa si borras un archivo? | SYMLINK
Aquí cambia todo:
```bash
ln -s archivo.txt simb.txt
```

Proceso hace:
```bash
open("simb.txt")
```
El kernel:
  1. Lee el symlink
  2. Resuelve "archivo.txt"
  3. Abre el inodo real
  4. fd apunta al inodo

> El symlink solo se usa al abrir

**Después de abrir:**
Si borras: `rm archivo.txt`
  * El proceso no se entera
  * Sigue leyendo/escribiendo
  * Porque ya tiene el inodo abierto

Pero si otro proceso intenta abrir el symlink `cat simb.txt`, ocurre error ya que el archivo no existe.

Porque:
  * El symlink apunta a una ruta rota.
  * El kernel no puede resolverla


#### Tips
Linux no borra archivos, borra nombres

Y libera datos solo cuando:
```bash
link count == 0
AND
open file descriptors == 0
```

El link count es un contador dentro del inodo que indica: Cuántas entradas de directorio apuntan a este inodo


#### Comprensión de errores de admins
Muchos creen:
  > “Si lo borro, ya no existe”

Tú sabes:
  > “Si alguien lo abrió, sigue ahí”